# Synthesize Adult

### Vanilla

In [ ]:
# ensure that any code changes are immediately reflected
%reload_ext autoreload
%autoreload 2
# load core modules
import os
from mostly_engine.core import split, encode, analyze, train, generate
# init logging
from mostly_engine.core.log.logging import init_logging
init_logging(log_file_path='engine.log')

In [ ]:
work_dir = 'adult-ws'
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
os.chdir(work_dir)

In [ ]:
tgt_data_path, _ = split('../adult_original_2k.csv.gz')
analyze()
encode()
train()
syn_data_path = generate(generation_size=100_000)

In [ ]:
import pandas as pd
tgt = pd.concat([pd.read_parquet(fn) for fn in tgt_data_path.glob("*.parquet")])
syn = pd.concat([pd.read_parquet(fn) for fn in syn_data_path.glob("*.parquet")])

In [ ]:
syn.to_csv('../adult_synthetic.csv.gz', index=False)

### Rules

In [4]:
import pandas as pd
df = pd.read_csv('../adult_original_2k.csv.gz')

In [30]:
rule1 = pd.merge(
    pd.merge(
        df['education'].drop_duplicates(),
        df['education-num'].drop_duplicates(),
        how='cross',
    ),
    df[['education', 'education-num']].drop_duplicates().assign(valid=True),
    how='left',
)
rule1 = rule1.loc[rule1.valid.isna(),['education', 'education-num']]
rule1

,education,education-num
1,HS-grad,10
2,HS-grad,13
3,HS-grad,15
4,HS-grad,7
5,HS-grad,11
...,...,...
250,Preschool,5
251,Preschool,2
252,Preschool,3
253,Preschool,6


In [28]:
rule2 = pd.merge(
    pd.Series([17, 18, 19, 20, 21, 22, 23, 24], name='age'),
    pd.Series(['Prof-school', 'Doctorate'], name='education'),
    how='cross',
)
rule2

,age,education
0,17,Prof-school
1,17,Doctorate
2,18,Prof-school
3,18,Doctorate
4,19,Prof-school
5,19,Doctorate
6,20,Prof-school
7,20,Doctorate
8,21,Prof-school
9,21,Doctorate


In [29]:
rule3 = pd.merge(
    pd.Series(['Husband', 'Wife'], name='relationship'),
    pd.Series(['Never-married', 'Divorced', 'Married-spouse-absent', 'Widowed', 'Seperated'], name='marital-status'),
    how='cross',
)
rule3

,relationship,marital-status
0,Husband,Never-married
1,Husband,Divorced
2,Husband,Married-spouse-absent
3,Husband,Widowed
4,Husband,Seperated
5,Wife,Never-married
6,Wife,Divorced
7,Wife,Married-spouse-absent
8,Wife,Widowed
9,Wife,Seperated
